In [28]:
alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
ADFGVX = {0: 'A', 1: 'D', 2: 'F', 3: 'G', 4: 'V', 5: 'X'}
# message = 'Vuong Van Duy'
message = 'My name is Vuong Van Duy from group 1362'
substitution_key = 'SUBSTITUTION'
transposition_key = 'TRANSPOSITION'

In [29]:
def set_substitution_key(substitution_key):
  res = ''

  for char in substitution_key:
    if char not in res:
      res += char

  return res

In [30]:
def get_order_char(key_word):
     key_word_sorted = sorted(key_word)
     order_char = []
     for i in key_word:
        index = key_word_sorted.index(i)
        order_char.append(index + 1)
        key_word_sorted[index] = '_'
         
     return order_char

In [31]:
get_order_char(transposition_key)

[12, 9, 1, 4, 10, 8, 6, 11, 2, 13, 3, 7, 5]

In [32]:
# đổi chỗ hai cột trong mảng hai chiều
def swap_column(matrix, i, j):
    for row in matrix:
        row[i], row[j] = row[j], row[i]
    return matrix

In [33]:
def get_substitution_matrix(substitution_key):
    substitution_key = set_substitution_key(substitution_key.upper())
    chars = list(substitution_key)
    for char in alphabet:
        if char not in substitution_key:
            chars.append(char)
    substitution_matrix = []
    
    for i in range(6):
        substitution_matrix.append(chars[i*6:i*6+6])
    
    return substitution_matrix

In [34]:
get_substitution_matrix(substitution_key)

[['S', 'U', 'B', 'T', 'I', 'O'],
 ['N', 'A', 'C', 'D', 'E', 'F'],
 ['G', 'H', 'J', 'K', 'L', 'M'],
 ['P', 'Q', 'R', 'V', 'W', 'X'],
 ['Y', 'Z', '0', '1', '2', '3'],
 ['4', '5', '6', '7', '8', '9']]

In [35]:
import pandas as pd
df = pd.DataFrame(get_substitution_matrix(substitution_key))

# Lưu vào file Excel
df.to_excel("output/output_subs_ADFGVX.xlsx", index=False, header=False)

In [36]:
def get_index_elem_arr2D(arr2D, elem):
    for i in range(len(arr2D)):
        for j in range(len(arr2D[i])):
            if arr2D[i][j] == elem:
                return (i, j)
    return -1

In [37]:
def encrypt_substitution(message, substitution_key):
    substitution_matrix = get_substitution_matrix(substitution_key)
    
    message = message.upper()
    for char in message:
        if char not in alphabet:
            message = message.replace(char, '')
    
    cipher_text = ''
    for char in message:
        row, col = get_index_elem_arr2D(substitution_matrix, char)
        cipher_text += ADFGVX[row] + ADFGVX[col]
        
    return cipher_text

In [38]:
cipher_substitution_text = encrypt_substitution(message, substitution_key)
print(cipher_substitution_text)

FXVADADDFXDVAVAAGGADAXDAFAGGDDDADGADVADXGFAXFXFAGFAXADGAVGVXXFVV


In [39]:
def matrix_before_transposition(message, substitution_key, transposition_key):
    cipher_text = encrypt_substitution(message, substitution_key)
    
    m = len(cipher_text)
    n = len(transposition_key)
    matrix = []
    for i in range(0, m, n):
        if i + n > m:
            matrix.append(list(cipher_text[i:i+n]) + [''] * (i + n - m))
        else:
            matrix.append(list(cipher_text[i:i+n]))
    
    return matrix

In [40]:
matrix_before_transpos = matrix_before_transposition(message, substitution_key, transposition_key)

matrix_before_transpos.insert(0, ['-'] * len(matrix_before_transpos[0]))
matrix_before_transpos.insert(0, transposition_key)
matrix_before_transpos.insert(0, get_order_char(transposition_key))
df = pd.DataFrame(matrix_before_transpos)
df.to_excel("output/output_before_transpos_ADFGVX.xlsx", index=False, header=False)

In [41]:
def encrypt_transposition_matrix(message, substitution_key, transposition_key):
    matrix = matrix_before_transposition(message, substitution_key, transposition_key)
    transposition_key_sorted = list(transposition_key)
    n = len(transposition_key)    
    order_char = get_order_char(transposition_key)

    # selection sort algorithm 
    for i in range(n - 1):
        min_ord = order_char[i]
        index = i
        for j in range(i + 1, n):
            if min_ord > order_char[j]:
                min_ord = order_char[j]
                index = j

        order_char[i], order_char[index] = order_char[index], order_char[i]
        transposition_key_sorted[i], transposition_key_sorted[index] = transposition_key_sorted[index], transposition_key_sorted[i]
        swap_column(matrix, i, index)
        
    return matrix, transposition_key_sorted

In [42]:
transposition_matrix, transposition_key_sorted = encrypt_transposition_matrix(message, substitution_key, transposition_key)

transposition_matrix.insert(0, ['-'] * len(transposition_matrix[0]))
transposition_matrix.insert(0, transposition_key_sorted)
transposition_matrix.insert(0, get_order_char(transposition_key_sorted))

for i in range(len(transposition_matrix)):
    for j in range(len(transposition_matrix[0])):
        if not transposition_matrix[i][j]:
            print('-', end=' ')
        else:
            print(transposition_matrix[i][j], end=' ')
    print()

1 2 3 4 5 6 7 8 9 10 11 12 13 
A I I N N O O P R S S T T 
- - - - - - - - - - - - - 
V F D A A D V A X D D F X 
A X A G A D F A A G A V D 
D A V D D D A A G D G G D 
F A F A X X A F G X F X G 
G X V A - V V G D V X A F 


In [43]:
df = pd.DataFrame(transposition_matrix)

# Lưu vào file Excel
df.to_excel("output/output_transpos_ADFGVX.xlsx", index=False, header=False)

In [44]:
def encrypt_ADGVFX(message, substitution_key, transposition_key, mode=False):
    matrix, _ = encrypt_transposition_matrix(message, substitution_key, transposition_key)
    cipher_text = ''.join(''.join(col) for col in zip(*matrix))
    return cipher_text
    

In [45]:
cipher_text = encrypt_ADGVFX(message, substitution_key, transposition_key)
print(cipher_text)

VADFGFXAAXDAVFVAGDAAAADXDDDXVVFAAVAAAFGXAGGDDGDXVDAGFXFVGXAXDDGF


In [46]:
def restore_transposition_matrix(cipher_text, transposition_key):
    num_cols = len(transposition_key)
    num_rows = (len(cipher_text) + num_cols - 1) // num_cols
    num_chars_last_row = (len(cipher_text) - 1) % num_cols + 1
    order_char = get_order_char(transposition_key)
    chars_last_row = order_char[0:num_chars_last_row]
    
    matrix = [[''] * num_cols for _ in range(num_rows)]
    index = 0
    for j in range(num_cols):
        for i in range(num_rows):
            if index < len(cipher_text):
                if i == num_rows - 1 and num_chars_last_row > 0:
                    if (j + 1) in chars_last_row:
                        num_chars_last_row -= 1
                        chars_last_row.remove(j + 1)
                    else:
                        continue
               
                matrix[i][j] = cipher_text[index]
                index += 1
    
    return matrix

In [47]:
transposition_matrix_restore = restore_transposition_matrix(cipher_text, transposition_key)

for i in range(len(transposition_matrix_restore)):
    for j in range(len(transposition_matrix_restore[0])):
        if not transposition_matrix_restore[i][j]:
            print('-', end=' ')
        else:
            print(transposition_matrix_restore[i][j], end=' ')
    print()

V F D A A D V A X D D F X 
A X A G A D F A A G A V D 
D A V D D D A A G D G G D 
F A F A X X A F G X F X G 
G X V A - V V G D V X A F 


In [48]:
def decrypt_substitution(cipher_text, transposition_key):
    transposition_matrix = restore_transposition_matrix(cipher_text, transposition_key)
    n = len(transposition_key)
    order_char = get_order_char(transposition_key)
    
    trace = []
    for i in range(n - 1):
        min_ord = order_char[i]
        index = i
        for j in range(i + 1, n):
            if min_ord > order_char[j]:
                min_ord = order_char[j]
                index = j
        if index != i:
            trace.append((i, index))
            order_char[i], order_char[index] = order_char[index], order_char[i]
    
    for i, index in trace[::-1]:
        swap_column(transposition_matrix, i, index)
        order_char[i], order_char[index] = order_char[index], order_char[i]
    
    return ''.join(''.join(row) for row in transposition_matrix)

In [49]:
cipher_substitution = decrypt_substitution(cipher_text, transposition_key)
print(cipher_substitution)

FXVADADDFXDVAVAAGGADAXDAFAGGDDDADGADVADXGFAXFXFAGFAXADGAVGVXXFVV


In [50]:
def decrypt_ADGVFX(cipher_text, substitution_key, transposition_key, mode=False):
    cipher_substitution = decrypt_substitution(cipher_text, transposition_key)
    substitution_matrix = get_substitution_matrix(substitution_key)
    
    plain_text = ''
    keys = list(ADFGVX.values())
    for i in range(0, len(cipher_substitution), 2):
        row = keys.index(cipher_substitution[i])
        col = keys.index(cipher_substitution[i + 1])
        plain_text += substitution_matrix[row][col]
    
    return plain_text

In [51]:
plain_text = decrypt_ADGVFX(cipher_text, substitution_key, transposition_key)
print(plain_text)

MYNAMEISVUONGVANDUYFROMGROUP1362


In [52]:
# in toàn bộ dữ liệu ra file text. 
substitution_matrix = get_substitution_matrix(substitution_key)
substitution_matrix.insert(0, ['*']*6)
substitution_matrix.insert(0, ['A', 'D', 'F', 'G', 'V', 'X'])
mat = []

with open('output_cipher/output_ADFGVX.txt', 'w') as f:
    f.write(f'Message: {message}\n')
    f.write(f'Substitution key: {substitution_key}\n')
    f.write(f'Transposition key: {transposition_key}\n')
    # in ma trận thay thế ra file
    f.write('\nSubstitution matrix:\n')
    
    for i, row in enumerate(substitution_matrix):
        if i == 0 or i == 1:
            row = [' ', ' '] + row
        else:
            row = [ADFGVX[i - 2], '*'] + row
        row = [char if char else '-' for char in row]
        mat.append(row)
        f.write(' '.join(row) + '\n')
    
    f.write('\n')
    # in từ mã sau khi thay thế ra file
    f.write(f'Cipher substitution text: {cipher_substitution_text}\n')
    # in ma trận trước khi hoán vị ra file
    f.write('\nMatrix before transposition:\n')
    for row in matrix_before_transpos:
        row = [str(char) if char else '-' for char in row]
        f.write(' '.join(row) + '\n')
    # in ma trận hoán vị ra file
    f.write('\nTransposition matrix:\n')
    for row in transposition_matrix:
        row = [str(char) if char else '-' for char in row]
        f.write(' '.join(row) + '\n')
    f.write(f'\nCipher text: {cipher_text}\n')
    f.write(f'\nPlain text: {plain_text}\n')